## Google Colab Rust Setup

The following cell is used to set up a Rust environment on Colab. Don't execute it locally!

Many thanks to [`mateusvmv`](https://github.com/mateusvmv) for this hack in [`gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3`](https://gist.github.com/korakot/ae95315ea6a3a3b33ee26203998a59a3?permalink_comment_id=4715636#gistcomment-4715636).

In [ ]:
# This script sets up and spins up a Jupyter Notebook environment with a Rust kernel using Nix and IPC Proxy. 
!wget -qO- https://gist.github.com/wiseaidev/2af6bef753d48565d11bcd478728c979/archive/3f6df40db09f3517ade41997b541b81f0976c12e.tar.gz | tar xvz --strip-components=1
!bash setup_evcxr_kernel.sh

## Install Required Dependencies

In [2]:
:dep plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
// or
// :dep plotters = { git = "https://github.com/plotters-rs/plotters" }

In [3]:
:dep ndarray = {version = "0.15.6"}
// or
// :dep ndarray = { git = "https://github.com/rust-ndarray/ndarray" }

In [4]:
:dep ndarray-rand = {version = "0.14.0"}
// or
// :dep ndarray-rand = { git = "https://github.com/rust-ndarray/ndarray/tree/ndarray-rand-0.14.0" }

### Import Modules

In [5]:
use plotters::prelude::*;
use ndarray::prelude::*;
use plotters::element::ErrorBar;
use ndarray::{array, Array, s};
use ndarray_rand::RandomExt;
use ndarray_rand::rand_distr::Uniform;

<hr />

## Single-Line Plots

### 1. Using Arrays

In [6]:
evcxr_figure((640, 240), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_2d(0f32..7f32, 0f32..7f32)?;

    let x_axis = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0];

    chart.draw_series(LineSeries::new(
        x_axis.map(|x| (x, x)),
        &RED,
    ))?;
    Ok(())
}).style("width:100%")

### 2. Using Ndarray

In [7]:
evcxr_figure((640, 240), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_2d(0f32..7f32, 0f32..7f32)?;
    
    let x_axis = Array::range(1., 7., 1.);
    
    chart.draw_series(LineSeries::new(
        x_axis.into_raw_vec().into_iter().map(|x| (x, x)),
        &RED,
    ))?;
    Ok(())
}).style("width:100%")

### Cubic Function

In [8]:
let points_coordinates: Vec<(f32, f32)> = {
    let x_axis = vec![1.0, 2.0, 3.0, 4.0, 5.0, 6.0];
    let cubic: Vec<f32> = x_axis.iter().map(|x| i32::pow(*x as i32, 3) as f32).collect::<Vec<f32>>();
    x_axis.into_iter().zip(cubic).collect()
};
points_coordinates

[(1.0, 1.0), (2.0, 8.0), (3.0, 27.0), (4.0, 64.0), (5.0, 125.0), (6.0, 216.0)]

In [9]:
evcxr_figure((640, 240), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_2d(0f32..7f32, 0f32..220f32)?;

    chart.draw_series(LineSeries::new(
        points_coordinates.iter().map(|(x, y)| (*x, *y)),
        &RED,
    ))?;
    Ok(())
}).style("width:100%")

<hr />

## Multiline Plots

### 1. Using Vectors

In [10]:
let linear_coordinates: Vec<(f32, f32)> = {
    let x_axis = vec![1.0, 2.0, 3.0, 4.0, 5.0, 6.0];
    let linear: Vec<f32> = x_axis.iter().map(|x| *x).collect::<Vec<f32>>();
    x_axis.into_iter().zip(linear).collect()
};
let quadratic_coordinates: Vec<(f32, f32)> = {
    let x_axis = vec![1.0, 2.0, 3.0, 4.0, 5.0, 6.0];
    let quadratic: Vec<f32> = x_axis.iter().map(|x| i32::pow(*x as i32, 2) as f32).collect::<Vec<f32>>();
    x_axis.into_iter().zip(quadratic).collect()
};
let cubic_coordinates: Vec<(f32, f32)> = {
    let x_axis = vec![1.0, 2.0, 3.0, 4.0, 5.0, 6.0];
    let cubic: Vec<f32> = x_axis.iter().map(|x| i32::pow(*x as i32, 3) as f32).collect::<Vec<f32>>();
    x_axis.into_iter().zip(cubic).collect()
};
quadratic_coordinates

[(1.0, 1.0), (2.0, 4.0), (3.0, 9.0), (4.0, 16.0), (5.0, 25.0), (6.0, 36.0)]

In [11]:
evcxr_figure((640, 240), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_2d(0f32..7f32, 0f32..220f32)?;

    chart.draw_series(LineSeries::new(
        linear_coordinates.iter().map(|(x, y)| (*x, *y)),
        &RED,
    ))?;

    chart.draw_series(LineSeries::new(
        quadratic_coordinates.iter().map(|(x, y)| (*x, *y)),
        &GREEN,
    ))?;

    chart.draw_series(LineSeries::new(
        cubic_coordinates.iter().map(|(x, y)| (*x, *y)),
        &BLUE,
    ))?;

    Ok(())
}).style("width:100%")

### 2. Using Ndarray

In [12]:
let linear_coordinates: Vec<(f32, f32)> = {
    let x_y_axes = array!([[1., 2., 3., 4.], [1., 2., 3., 4.]]);
    let x_axis: Vec<f32> = x_y_axes.slice(s![0, 0, ..]).to_vec();
    let y_axis: Vec<f32> = x_y_axes.slice(s![0, 1, ..]).to_vec();
    x_axis.into_iter().zip(y_axis).collect()
};
let quadratic_coordinates: Vec<(f32, f32)> = {
    let x_y_axes = array!([[1., 2., 3., 4.], [1., 4., 9., 16.]]);
    let x_axis: Vec<f32> = x_y_axes.slice(s![0, 0, ..]).to_vec();
    let y_axis: Vec<f32> = x_y_axes.slice(s![0, 1, ..]).to_vec();
    x_axis.into_iter().zip(y_axis).collect()
};
quadratic_coordinates

[(1.0, 1.0), (2.0, 4.0), (3.0, 9.0), (4.0, 16.0)]

In [13]:
evcxr_figure((640, 240), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_2d(0f32..5f32, 0f32..17f32)?;

    chart.draw_series(LineSeries::new(
        linear_coordinates.iter().map(|(x, y)| (*x, *y)),
        &RED,
    ))?;

    chart.draw_series(LineSeries::new(
        quadratic_coordinates.iter().map(|(x, y)| (*x, *y)),
        &GREEN,
    ))?;

    Ok(())
}).style("width:100%")

### 3. Using Ndarray-Rand

In [14]:
let mut random_samples: Vec<(f32, f32)> = {
    let x_y_axes = Array::random((2, 50), Uniform::new(0., 1.));
    let x_axis: Vec<f32> = x_y_axes.slice(s![0, ..]).to_vec();
    let y_axis: Vec<f32> = x_y_axes.slice(s![1, ..]).to_vec();
    x_axis.into_iter().zip(y_axis).collect()
};
random_samples.sort_by(|(a, b), (c, d)| a.partial_cmp(c).unwrap());
random_samples[..10]

[(0.0054428577, 0.77135944), (0.006298661, 0.3798182), (0.012778878, 0.61504066), (0.012791395, 0.9146923), (0.016703725, 0.7325721), (0.016977787, 0.85884607), (0.023496985, 0.098095536), (0.06909549, 0.8486662), (0.071852684, 0.32104456), (0.09480286, 0.65017307)]

In [15]:
evcxr_figure((640, 240), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;

    chart.draw_series(LineSeries::new(
        random_samples.iter().map(|(x, y)| (*x, *y)),
        &RED,
    ))?;

    Ok(())
}).style("width:100%")

<hr />

## Grid, Axes, and Labels

### 1. Add a Mesh/Grid

In [16]:
evcxr_figure((640, 240), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;

    chart.configure_mesh().draw()?;

    Ok(())
}).style("width:100%")

### 2. Add Labels

In [17]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 1.);")
        .y_desc("y = f(x)")
        .draw()?;
    
    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 1.);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

### 3. Legend

In [18]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..14f32)?;
    
    let x = Array::range(1., 7., 0.1);
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x)),
        &RED
    )).unwrap()
        .label("y = x")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    
    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x * 2.0)),
        &GREEN
    )).unwrap()
        .label("y = 2 * x")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));

    chart.configure_series_labels()
        .background_style(&WHITE)
        .border_style(&BLACK)
        .draw()?;

    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="50,429 59,426 69,423 79,420 89,417 99,414 108,411 118,408 128,405 138,402 148,398 157,395 167,392 177,389 187,386 197,383 207,380 216,377 226,374 236,371 246,367 256,364 265,361 275,358 285,355 295,352 305,349 315,346 324,343 334,340 344,336 354,333 364,330 373,327 383,324 393,321 403,318 413,315 423,312 432,309 442,305 452,302 462,299 472,296 481,293 491,290 501,287 511,284 521,281 531,278 540,274 550,271 560,268 570,265 580,262 589,259 599,256 609,253 619,250 629,247 "/>
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="50,398 59,392 69,386 79,380 89,374 99,367 108,361 118,355 128,349 138,343 148,336 157,330 167,324 177,318 187,312 197,305 207,299 216,293 226,287 236,281 246,274 256,268 265,262 275,256 285,250 295,243 305,237 315,231 324,225 334,219 344,212 354,206 364,200 373,194 383,188 393,181 403,175 413,169 423,163 432,157 442,150 452,144 462,138 472,132 481,126 491,119 501,113 511,107 521,101 531,95 540,88 550,82 560,76 570,70 580,64 589,57 599,51 609,45 619,39 629,33 "/>
 
 
 
y = x
 
 
y = 2 * x

### 4. Legend Position

In [19]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..14f32)?;
    
    let x = Array::range(1., 7., 0.1);
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x)),
        &RED
    )).unwrap()
        .label("y = x")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));
    
    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x * 2.0)),
        &GREEN
    )).unwrap()
        .label("y = 2 * x")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));

    chart.configure_series_labels()
        .position(SeriesLabelPosition::UpperMiddle)
        .background_style(&WHITE)
        .border_style(&BLACK)
        .draw()?;

    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="50,429 59,426 69,423 79,420 89,417 99,414 108,411 118,408 128,405 138,402 148,398 157,395 167,392 177,389 187,386 197,383 207,380 216,377 226,374 236,371 246,367 256,364 265,361 275,358 285,355 295,352 305,349 315,346 324,343 334,340 344,336 354,333 364,330 373,327 383,324 393,321 403,318 413,315 423,312 432,309 442,305 452,302 462,299 472,296 481,293 491,290 501,287 511,284 521,281 531,278 540,274 550,271 560,268 570,265 580,262 589,259 599,256 609,253 619,250 629,247 "/>
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="50,398 59,392 69,386 79,380 89,374 99,367 108,361 118,355 128,349 138,343 148,336 157,330 167,324 177,318 187,312 197,305 207,299 216,293 226,287 236,281 246,274 256,268 265,262 275,256 285,250 295,243 305,237 315,231 324,225 334,219 344,212 354,206 364,200 373,194 383,188 393,181 403,175 413,169 423,163 432,157 442,150 452,144 462,138 472,132 481,126 491,119 501,113 511,107 521,101 531,95 540,88 550,82 560,76 570,70 580,64 589,57 599,51 609,45 619,39 629,33 "/>
 
 
 
y = x
 
 
y = 2 * x

<hr />

## Colors and Markers

#### 1. Colors

In [20]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..14f32)?;
    
    let x = Array::range(1., 7., 0.1);
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x)),
        &RGBColor(0,0,255) // red: 0, green: 0, blue: 255 -> the color is blue 
    ))?;
    
    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="50,429 59,426 69,423 79,420 89,417 99,414 108,411 118,408 128,405 138,402 148,398 157,395 167,392 177,389 187,386 197,383 207,380 216,377 226,374 236,371 246,367 256,364 265,361 275,358 285,355 295,352 305,349 315,346 324,343 334,340 344,336 354,333 364,330 373,327 383,324 393,321 403,318 413,315 423,312 432,309 442,305 452,302 462,299 472,296 481,293 491,290 501,287 511,284 521,281 531,278 540,274 550,271 560,268 570,265 580,262 589,259 599,256 609,253 619,250 629,247 "/>

In [21]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..14f32)?;
    
    let x = Array::range(1., 7., 0.1);
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x)),
        &HSLColor(5.0, 1.0, 0.7)
    ))?;
    
    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
<polyline fill="none" opacity="1" stroke="#FF6666" stroke-width="1" points="50,429 59,426 69,423 79,420 89,417 99,414 108,411 118,408 128,405 138,402 148,398 157,395 167,392 177,389 187,386 197,383 207,380 216,377 226,374 236,371 246,367 256,364 265,361 275,358 285,355 295,352 305,349 315,346 324,343 334,340 344,336 354,333 364,330 373,327 383,324 393,321 403,318 413,315 423,312 432,309 442,305 452,302 462,299 472,296 481,293 491,290 501,287 511,284 521,281 531,278 540,274 550,271 560,268 570,265 580,262 589,259 599,256 609,253 619,250 629,247 "/>

#### 2. Markers

In [22]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..8f32)?;
    
    let x = Array::range(1., 7., 0.1);
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x)),
        &RED
    ))?;
    chart.draw_series(x.map(|x| {
        EmptyElement::at((*x, *x))
        + Cross::new((0, 0), 2, GREEN) // coordinates relative to EmptyElement
//         + Circle::new((0, 0), 2, GREEN) // uncomment one of these markers
//         + Pixel::new((0, 0), GREEN)
//         + TriangleMarker::new((0, 0), 2, GREEN)
    }))?;
    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="50,429 59,424 69,418 79,412 89,406 99,401 108,395 118,389 128,383 138,378 148,372 157,366 167,360 177,355 187,349 197,343 207,337 216,332 226,326 236,320 246,314 256,309 265,303 275,297 285,291 295,286 305,280 315,274 324,268 334,263 344,257 354,251 364,245 373,240 383,234 393,228 403,222 413,216 423,211 432,205 442,199 452,193 462,188 472,182 481,176 491,170 501,165 511,159 521,153 531,147 540,142 550,136 560,130 570,124 580,119 589,113 599,107 609,101 619,96 629,90 "/>

In [23]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..8f32)?;
    
    let x = Array::range(1., 7., 0.1);
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x)),
        &RED
    ).point_size(2))?;

    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="50,429 59,424 69,418 79,412 89,406 99,401 108,395 118,389 128,383 138,378 148,372 157,366 167,360 177,355 187,349 197,343 207,337 216,332 226,326 236,320 246,314 256,309 265,303 275,297 285,291 295,286 305,280 315,274 324,268 334,263 344,257 354,251 364,245 373,240 383,234 393,228 403,222 413,216 423,211 432,205 442,199 452,193 462,188 472,182 481,176 491,170 501,165 511,159 521,153 531,147 540,142 550,136 560,130 570,124 580,119 589,113 599,107 609,101 619,96 629,90 "/>

In [24]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..8f32)?;
    
    let x = Array::range(1., 7., 0.1);
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x)),
        RED.filled()
    ).point_size(2))?;

    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="50,429 59,424 69,418 79,412 89,406 99,401 108,395 118,389 128,383 138,378 148,372 157,366 167,360 177,355 187,349 197,343 207,337 216,332 226,326 236,320 246,314 256,309 265,303 275,297 285,291 295,286 305,280 315,274 324,268 334,263 344,257 354,251 364,245 373,240 383,234 393,228 403,222 413,216 423,211 432,205 442,199 452,193 462,188 472,182 481,176 491,170 501,165 511,159 521,153 531,147 540,142 550,136 560,130 570,124 580,119 589,113 599,107 609,101 619,96 629,90 "/>

#### 3. Colors + Markers + Legend

In [25]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Plot Demo", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..342f32)?;
    
    let x = Array::range(1., 7., 0.1);
    
    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, *x)),
        RED.filled()
    ).point_size(2)).unwrap()
        .label("y = x")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &RED));

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, (*x).powi(3))),
        BLUE
    ).point_size(2)).unwrap()
        .label("y = x ^ 3")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &BLUE));

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, (*x).powi(2))),
        &GREEN
    )).unwrap()
        .label("y = x ^ 2")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));

    chart.draw_series(x.map(|x| {
        EmptyElement::at((*x, (*x).powi(2)))
        + Cross::new((0, 0), 2, WHITE) // coordinates relative to EmptyElement
//         + Circle::new((0, 0), 2, GREEN) // uncomment one of these markers
//         + Pixel::new((0, 0), GREEN)
//         + TriangleMarker::new((0, 0), 2, GREEN)
    }))?;

    chart.configure_series_labels()
        .background_style(&WHITE)
        .border_style(&BLACK)
        .draw()?;

    Ok(())
}).style("width: 60%")

Plot Demo
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
50.0
 
 
 
100.0
 
 
 
150.0
 
 
 
200.0
 
 
 
250.0
 
 
 
300.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="50,429 59,429 69,429 79,429 89,429 99,429 108,429 118,429 128,429 138,428 148,428 157,428 167,428 177,428 187,428 197,428 207,428 216,427 226,427 236,427 246,427 256,427 265,427 275,427 285,427 295,427 305,426 315,426 324,426 334,426 344,426 354,426 364,426 373,426 383,425 393,425 403,425 413,425 423,425 432,425 442,425 452,425 462,425 472,424 481,424 491,424 501,424 511,424 521,424 531,424 540,424 550,423 560,423 570,423 580,423 589,423 599,423 609,423 619,423 629,423 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="50,429 59,429 69,429 79,428 89,427 99,427 108,426 118,425 128,424 138,423 148,421 157,420 167,418 177,416 187,414 197,412 207,410 216,407 226,405 236,402 246,399 256,395 265,392 275,388 285,384 295,380 305,376 315,371 324,366 334,361 344,355 354,349 364,343 373,337 383,330 393,323 403,316 413,308 423,300 432,292 442,283 452,274 462,265 472,255 481,245 491,234 501,223 511,212 521,200 531,188 540,175 550,162 560,149 570,135 580,121 589,106 599,91 609,75 619,59 629,42 "/>
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="50,429 59,429 69,429 79,429 89,428 99,428 108,428 118,427 128,427 138,426 148,426 157,425 167,425 177,424 187,424 197,423 207,423 216,422 226,421 236,421 246,420 256,419 265,419 275,418 285,417 295,416 305,415 315,415 324,414 334,413 344,412 354,411 364,410 373,409 383,408 393,407 403,406 413,405 423,403 432,402 442,401 452,400 462,399 472,397 481,396 491,395 501,394 511,392 521,391 531,390 540,388 550,387 560,385 570,384 580,382 589,381 599,379 609,378 619,376 629,374 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = x
 
 
y = x ^ 3
 
 
y = x ^ 2

<hr />

## Subplots

In [26]:
let linear_coordinates: Vec<(f32, f32)> = {
    let x_y_axes = array!([[1., 2., 3., 4.], [1., 2., 3., 4.]]);
    let x_axis: Vec<f32> = x_y_axes.slice(s![0, 0, ..]).to_vec();
    let y_axis: Vec<f32> = x_y_axes.slice(s![0, 1, ..]).to_vec();
    x_axis.into_iter().zip(y_axis).collect()
};
let quadratic_coordinates: Vec<(f32, f32)> = {
    let x_y_axes = array!([[1., 2., 3., 4.], [1., 4., 9., 16.]]);
    let x_axis: Vec<f32> = x_y_axes.slice(s![0, 0, ..]).to_vec();
    let y_axis: Vec<f32> = x_y_axes.slice(s![0, 1, ..]).to_vec();
    x_axis.into_iter().zip(y_axis).collect()
};
quadratic_coordinates

[(1.0, 1.0), (2.0, 4.0), (3.0, 9.0), (4.0, 16.0)]

### 1. Using split_evenly

In [27]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let sub_areas = root.split_evenly((1,2)); // 1x2 grid ( 1 row, 2 columns)

    let graphs = vec![
        ("y = x", linear_coordinates.clone(), &RED),
        ("y= x ^ 2", quadratic_coordinates.clone(), &GREEN),
    ];

    for ((idx, area), graph) in (1..).zip(sub_areas.iter()).zip(graphs.iter()) {
        let mut chart = ChartBuilder::on(&area)
            .caption(graph.0, ("Arial", 15).into_font())
            .margin(50)
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0f32..5f32, 0f32..17f32)?;

        chart.draw_series(LineSeries::new(
            graph.1.iter().map(|(x, y)| (*x, *y)),
            graph.2,
        )).unwrap()
            .label(graph.0)
            .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));

        chart.configure_mesh()
            .y_labels(10)
            .light_line_style(&TRANSPARENT)
            .disable_x_mesh()
            .x_desc("x = Array::range(1., 7., 0.1);")
            .y_desc(graph.0)
            .draw()?;
    }

    Ok(())
}).style("width:100%")

y = x
 
 
 
y = x
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0
 
 
 
y= x ^ 2
 
 
 
y= x ^ 2
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0

### 2. Using split_horizontally

Many thanks to [Ali](https://github.com/AliMMehr) for fixing this issue in [`plotters/issues/486`](https://github.com/plotters-rs/plotters/issues/486)

In [33]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let sub_areas = root.split_horizontally(320); // 320 is the width of the left section in pixels

    let graphs = vec![
        ("y = x", linear_coordinates.clone(), &RED, sub_areas.0),
        ("y= x ^ 2", quadratic_coordinates.clone(), &GREEN, sub_areas.1),
    ];

    for (idx, graph) in (1..).zip(graphs.iter()) {
        let mut chart = ChartBuilder::on(&graph.3)
            .margin(50)
            .caption(graph.0, ("Arial", 15).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0f32..5f32, 0f32..17f32)?;

        chart.draw_series(LineSeries::new(
            graph.1.iter().map(|(x, y)| (*x, *y)),
            graph.2,
        )).unwrap()
            .label(graph.0)
            .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));

        chart.configure_mesh()
            .y_labels(10)
            .light_line_style(&TRANSPARENT)
            .disable_x_mesh()
            .x_desc("x = Array::range(1., 7., 0.1);")
            .y_desc(graph.0)
            .draw()?;
    }

    Ok(())
}).style("width:100%")

y = x
 
 
 
y = x
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0
 
 
 
y= x ^ 2
 
 
 
y= x ^ 2
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
2.0
 
 
 
4.0
 
 
 
6.0
 
 
 
8.0
 
 
 
10.0
 
 
 
12.0
 
 
 
14.0
 
 
 
16.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0

<hr />

## Error Bars

### 1. Vertical Bars

In [29]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Vertical Error Bars Plot", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..50f32)?;
    
    let x = Array::range(1., 7., 0.1);

    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, (*x as f32).powi(2))),
        &GREEN
    )).unwrap()
        .label("y = x ^ 2")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));
  
    chart.draw_series(x.map(|x| {
        ErrorBar::new_vertical(*x, (*x as f32).powi(2) - 1.5, (*x as f32).powi(2), (*x as f32).powi(2) + 1.4, RED.filled(), 2)
    })).unwrap();

    chart.configure_series_labels()
        .background_style(&WHITE)
        .border_style(&BLACK)
        .draw()?;

    Ok(())
}).style("width: 100%")

Vertical Error Bars Plot
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
5.0
 
 
 
10.0
 
 
 
15.0
 
 
 
20.0
 
 
 
25.0
 
 
 
30.0
 
 
 
35.0
 
 
 
40.0
 
 
 
45.0
 
 
 
50.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="50,429 59,428 69,426 79,424 89,422 99,419 108,417 118,414 128,411 138,408 148,405 157,401 167,398 177,394 187,390 197,386 207,382 216,378 226,373 236,369 246,364 256,359 265,354 275,348 285,343 295,337 305,331 315,325 324,319 334,313 344,306 354,299 364,293 373,286 383,278 393,271 403,264 413,256 423,248 432,240 442,232 452,224 462,215 472,207 481,198 491,189 501,180 511,171 521,161 531,151 540,142 550,132 560,122 570,111 580,101 589,90 599,79 609,69 619,57 629,46 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = x ^ 2

### 2. Horizontal Bars

In [30]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Horizontal Error Bars Plot", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1f32..7f32, 1f32..50f32)?;
    
    let x = Array::range(1., 7., 0.1);

    chart.configure_mesh()
        .x_desc("x = Array::range(1., 7., 0.1);")
        .y_desc("y = f(x)")
        .draw()?;

    chart.draw_series(LineSeries::new(
        x.iter().map(|x| (*x, (*x as f32).powi(2))),
        &GREEN
    )).unwrap()
        .label("y = x ^ 2")
        .legend(|(x,y)| PathElement::new(vec![(x,y), (x + 20,y)], &GREEN));
  
    chart.draw_series(x.map(|x| {
        ErrorBar::new_horizontal((*x as f32).powi(2), *x - 0.3, *x, *x + 0.3, RED.filled(), 2)
    })).unwrap();

    chart.configure_series_labels()
        .background_style(&WHITE)
        .border_style(&BLACK)
        .draw()?;

    Ok(())
}).style("width: 100%")

Horizontal Error Bars Plot
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = f(x)
 
 
x = Array::range(1., 7., 0.1);
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
5.0
 
 
 
10.0
 
 
 
15.0
 
 
 
20.0
 
 
 
25.0
 
 
 
30.0
 
 
 
35.0
 
 
 
40.0
 
 
 
45.0
 
 
 
50.0
 
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
<polyline fill="none" opacity="1" stroke="#00FF00" stroke-width="1" points="50,429 59,428 69,426 79,424 89,422 99,419 108,417 118,414 128,411 138,408 148,405 157,401 167,398 177,394 187,390 197,386 207,382 216,378 226,373 236,369 246,364 256,359 265,354 275,348 285,343 295,337 305,331 315,325 324,319 334,313 344,306 354,299 364,293 373,286 383,278 393,271 403,264 413,256 423,248 432,240 442,232 452,224 462,215 472,207 481,198 491,189 501,180 511,171 521,161 531,151 540,142 550,132 560,122 570,111 580,101 589,90 599,79 609,69 619,57 629,46 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y = x ^ 2

<hr />

## Scatter Plot

In [31]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Uniform Distribution Scatter Plot", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f32..1f32, 0f32..1f32)?;

    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .y_labels(5)
        .x_label_formatter(&|x| format!("{:.1}", *x as f64 / 100.0))
        .y_label_formatter(&|y| format!("{}%", (*y * 100.0) as u32))
        .draw()?;

    let _ = chart.draw_series(random_samples.iter().map(|(x,y)| Circle::new((*x,*y), 3, GREEN.filled())));
    
    Ok(())
}).style("width:100%")

Uniform Distribution Scatter Plot
 
 
 
0%
 
 
 
50%
 
 
 
100%
 
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0
 
 
 
0.0

<hr />

## Histogram

In [32]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Histogram", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(0u32..100u32, 0f64..0.5f64)?;

    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .y_labels(5)
        .x_label_formatter(&|x| format!("{:.1}", *x as f64 / 100.0))
        .y_label_formatter(&|y| format!("{}%", (*y * 100.0) as u32))
        .draw()?;

    let hist = Histogram::vertical(&chart)
        .style(RED.filled())
        .margin(0)
        .data(random_samples.iter().map(|(x,_)| ((x*100.0) as u32, 0.01)));

    let _ = chart.draw_series(hist);
    
    Ok(())
}).style("width:100%")

Histogram
 
 
 
0%
 
 
 
20%
 
 
 
40%
 
 
 
 
0.0
 
 
 
0.1
 
 
 
0.2
 
 
 
0.3
 
 
 
0.4
 
 
 
0.5
 
 
 
0.6
 
 
 
0.7
 
 
 
0.8
 
 
 
0.9
 
 
 
1.0

---
---